## The following block of code should be placed in a folder which contains all of the videos taken of the particle in the liquid bath. The code is designed to track the object for all of the videos in the folder. The user must enter in a threshhold value (which will depend on the contrast and brightness of the object in the video), and the process_fraction (which is the percent of each video to process. It is set to 1 for the full video). Once you run the block of code, you will be prompted with selecting the bath's center, edge, and the center of the object. This will be used in order to create a mask of all of the image outside of the bath. This will assist in not accidently tracking extraneous objects in the background of the video

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
from glob import glob

# === USER CONFIGURATION ===
thresholdval = 220          # Threshold for binarization (pixels above this are considered object)
process_fraction = 1        # Fraction of frames to process; <1 for debugging smaller portion
video_ext = "*.MOV"         # Video file extension to look for

# === Find all .mov videos in the current directory ===
video_files = sorted(glob(video_ext))  # Get list of video files matching the pattern
if not video_files:
    print("No .mov files found in current directory.")
    exit()  # Exit if no videos found

# === Global variables for mouse input ===
clicks = []        # Stores the 3 clicks from user
center = None      # Center of bath (to be determined by first click)
radius = None      # Radius of bath (distance between first and second click)
target_click = None # Click location of the object to track
target_radius = None # Radius of target object (calculated later)

# === Mouse callback for first video only ===
def mouse_callback(event, x, y, flags, param):
    """
    Records mouse clicks from the user. Expects 3 clicks:
    1. Center of bath
    2. Edge of bath (to define radius)
    3. Target object to track
    """
    global clicks, target_click
    if event == cv2.EVENT_LBUTTONDOWN:
        clicks.append((x, y))  # Save click coordinates
        print(f"Click recorded at: ({x}, {y})")
        if len(clicks) == 3:
            cv2.destroyAllWindows()  # Close window once 3 clicks are done
            target_click = clicks[2]

# === STEP 1: Get crop window and target size from first video ===
first_video = video_files[0]  # Process first video for user input
cap = cv2.VideoCapture(first_video)
ret, frame = cap.read()  # Read first frame
if not ret:
    raise RuntimeError(f"Failed to read {first_video}")

# Get video dimensions and FPS
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)

# --- Let user click 3 times: center, edge, target object
cv2.namedWindow("Select Bath and Object (Click center, edge, then object)")
cv2.setMouseCallback("Select Bath and Object (Click center, edge, then object)", mouse_callback)

while True:
    temp_frame = frame.copy()
    # Draw visual feedback for clicks
    if len(clicks) >= 1:
        cv2.circle(temp_frame, clicks[0], 5, (0, 255, 0), -1)  # Center
    if len(clicks) >= 2:
        # Draw circle representing bath boundary
        cv2.circle(temp_frame, clicks[0], int(np.hypot(clicks[0][0] - clicks[1][0], clicks[0][1] - clicks[1][1])), (0, 255, 0), 2)
    if len(clicks) == 3:
        cv2.circle(temp_frame, clicks[2], 5, (255, 0, 0), -1)  # Target object

    cv2.imshow("Select Bath and Object (Click center, edge, then object)", temp_frame)
    if cv2.waitKey(1) & 0xFF == 27 or len(clicks) == 3:  # ESC or 3 clicks ends loop
        break

cv2.destroyAllWindows()

# Ensure all 3 clicks were recorded
if len(clicks) < 3:
    raise ValueError("Did not receive all 3 clicks.")

center = clicks[0]
edge = clicks[1]
target_click = clicks[2]

# --- Compute bath radius and crop window ---
radius = int(np.hypot(center[0] - edge[0], center[1] - edge[1]))  # Distance between center and edge click
x1 = max(center[0] - radius, 0)  # Crop window bounds
y1 = max(center[1] - radius, 0)
x2 = min(center[0] + radius, width)
y2 = min(center[1] + radius, height)
crop_w = x2 - x1
crop_h = y2 - y1

# --- Create bath mask ---
full_mask = np.zeros((height, width), dtype=np.uint8)
cv2.circle(full_mask, center, radius, 255, -1)  # White circle in black background representing bath

# --- Estimate target radius from first frame ---
masked = cv2.bitwise_and(frame, frame, mask=full_mask)  # Keep only bath area
cropped = masked[y1:y2, x1:x2]  # Crop to bath region
gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, thresholdval, 255, cv2.THRESH_BINARY)  # Binary threshold

# Find all contours in binary image
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Adjust target click coordinates relative to cropped area
adjusted_x = target_click[0] - x1
adjusted_y = target_click[1] - y1

# Determine the radius of the target object
target_radius = None
for contour in contours:
    if cv2.pointPolygonTest(contour, (adjusted_x, adjusted_y), False) >= 0:
        (_, _), r = cv2.minEnclosingCircle(contour)  # Fit minimum enclosing circle
        target_radius = r
        break

if target_radius is None:
    raise RuntimeError("Failed to find a contour near target click.")

print(f"\nBath radius: {radius} px")
print(f"Target object radius: {target_radius:.2f} px")

# === Radius filter limits for tracking ===
min_r = 0.8 * target_radius  # Minimum radius to accept
max_r = 1.2 * target_radius  # Maximum radius to accept

# === STEP 2: Loop through all videos ===
for video_path in video_files:
    print(f"\nProcessing {video_path}...")
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_process = int(total_frames * process_fraction)  # Fraction of frames

    base, ext = os.path.splitext(video_path)
    output_video = base + "_tracked.mp4"  # Output video path
    output_csv = base + "_tracked.csv"    # Output CSV path

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (crop_w, crop_h))  # Initialize output video
    positions = []  # Store tracking info
    frame_num = 0

    while frame_num < frames_to_process:
        ret, frame = cap.read()
        if not ret:
            break

        masked = cv2.bitwise_and(frame, frame, mask=full_mask)  # Apply bath mask
        cropped = masked[y1:y2, x1:x2]  # Crop to bath
        gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, thresholdval, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            # Sort contours by area (largest first)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)
            for contour in contours:
                (cx, cy), r = cv2.minEnclosingCircle(contour)  # Fit circle to contour
                if min_r <= r <= max_r:  # Accept only if radius within expected range
                    center_int = (int(round(cx)), int(round(cy)))
                    cv2.circle(cropped, center_int, int(r), (0, 255, 0), 2)  # Draw circle
                    cv2.circle(cropped, center_int, 2, (0, 0, 255), -1)       # Draw center point
                    positions.append({
                        'frame': frame_num,
                        'time': frame_num / fps,
                        'x': cx,
                        'y': cy,
                        'radius': r
                    })
                    break  # Stop after first valid contour

        out.write(cropped)  # Write annotated frame to output video
        frame_num += 1

    cap.release()
    out.release()

    # Save tracking data to CSV
    df = pd.DataFrame(positions)
    df.to_csv(output_csv, index=False, float_format="%.4f")
    print(f"Saved: {output_video}, {output_csv}")
